In [41]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [42]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-06 19:48:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.5’

book-crossings.zip. 100%[===================>]  24.88M  84.1MB/s    in 0.3s    

2025-06-06 19:48:10 (84.1 MB/s) - ‘book-crossings.zip.5’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [43]:
# Cargar dataframes
df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)

df_books = pd.read_csv(
    books_filename,
    encoding='ISO-8859-1',
    sep=';',
    header=0,
    names=['isbn', 'title', 'author', 'year', 'publisher', 'image_url_s', 'image_url_m', 'image_url_l'],
    usecols=['isbn', 'title'],
    dtype={'isbn': 'str', 'title': 'str'}
)


In [44]:
# Mostrar información básica
print(f"Total ratings originales: {df_ratings.shape[0]}")
print(f"Usuarios únicos originales: {df_ratings['user'].nunique()}")
print(f"Libros únicos originales: {df_ratings['isbn'].nunique()}")

# Definir umbrales
user_threshold = 200
book_threshold = 100

# Filtrar usuarios y libros según umbral
valid_users = df_ratings['user'].value_counts()[lambda x: x >= user_threshold].index
valid_books = df_ratings['isbn'].value_counts()[lambda x: x >= book_threshold].index

df_ratings_filtered = df_ratings[
    df_ratings['user'].isin(valid_users) & df_ratings['isbn'].isin(valid_books)
]

print(f"Filas después del filtrado: {df_ratings_filtered.shape[0]}")

# Merge para añadir títulos
df_ratings_filtered = pd.merge(df_ratings_filtered, df_books, on='isbn', how='inner')

# Crear matriz libro-usuario
book_user_matrix = df_ratings_filtered.pivot_table(index='title', columns='user', values='rating').fillna(0)

print("Forma de la matriz:", book_user_matrix.shape)

# Crear matriz dispersa
book_user_sparse = csr_matrix(book_user_matrix.values)

# Entrenar modelo KNN
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

print("Modelo entrenado correctamente.")



Total ratings originales: 1149780
Usuarios únicos originales: 105283
Libros únicos originales: 340556
Filas después del filtrado: 49781
Forma de la matriz: (673, 888)
Modelo entrenado correctamente.


In [45]:
# Función de recomendación
def get_recommends(book=""):
    if book not in book_user_matrix.index:
        return [book, []]

    book_index = book_user_matrix.index.get_loc(book)
    distances, indices = model.kneighbors(
        book_user_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6
    )

    recommended = []
    for i in range(1, 6):  # obtener 5 libros recomendados
        title = book_user_matrix.index[indices[0][i]]
        dist = float(distances[0][i])
        recommended.append([title, dist])

    # No ordenar, ya vienen ordenados
    print(f"Recomendaciones para {book}: {recommended}")
    return [book, recommended]


In [46]:
# Prueba y muestra resultados
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True', 'The Lovely Bones: A Novel']

    for i in range(5):
        if recommends[1][i][0] not in recommended_books:
            print(f"Libro no esperado: {recommends[1][i][0]}")
            test_pass = False
        # Puedes ajustar el margen según distancia esperada
        if not (0 <= recommends[1][i][1] <= 1):
            test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()

Recomendaciones para Where the Heart Is (Oprah's Book Club (Paperback)): [['The Lovely Bones: A Novel', 0.7234864234924316], ['I Know This Much Is True', 0.7677075266838074], ['The Surgeon', 0.7699410915374756], ['The Weight of Water', 0.7708583474159241], ["I'll Be Seeing You", 0.8016210794448853]]
You passed the challenge! 🎉🎉🎉🎉🎉
